In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data.plot_state_map import PlotStateMap

state_population_dict = {
    "CA": 39538223,
    "TX": 29145505,
    "FL": 21538187,
    "NY": 20201249,
    "PA": 13002700,
    "IL": 12812508,
    "OH": 11799448,
    "GA": 10711908,
    "NC": 10439388,
    "MI": 10077331
}

state_map = PlotStateMap(state_population_dict)
state_map.plot_map()


In [ ]:
df = pd.DataFrame(list(state_population_dict.items()), columns=["State", "Income"])
PlotStateMap(df, "My title", "Income").plot_map()